**Importing Libraries**

In [ ]:
import shutil
import os, sys, random
from glob import glob
import pandas as pd
from shutil import copyfile
from sklearn import preprocessing, model_selection
import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np
import os

In [ ]:
import wandb
wandb.login(key='f540abfade248550e8bc3abee9a5b1914d2ad798')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: giekoolis. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\gieko/.netrc


True

Refer [this blog](https://towardsai.net/p/computer-vision/yolo-v5-object-detection-on-a-custom-dataset) for more information. Its an excellent resource.  

**Cloning Official Repo**  


In [ ]:
# !git clone  'https://github.com/ultralytics/yolov5.git'
# !sed -i 's/PyYAML>=5.3.1/PyYAML==5.4.1/g' ./yolov5/requirements.txt
# !pip install -qr C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/requirements.txt

You should consider upgrading via the 'c:\users\gieko\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
# # Setting the model parameters
# shutil.copyfile('/content/tobacco_data.yaml', '/content/yolov5/tobacco_data.yaml') # copying the custom_dataset.yaml file to the project repo
# # setting number of classes to two (since the tobacco 800 dataset contains 2 classes, Logo & Signature)
# !sed -i 's/nc: 80/nc: 2/g' ./yolov5/models/yolov5x.yaml 
# !cat yolov5/tobacco_data.yaml

train: /content/data/images/train/
val: /content/data/images/valid/

nc: 2
names: ['DLLogo', 'DLSignature']


## Download dataset and prepare folder for all data

In [ ]:
def data_split(files, path_to_source, path_to_destination):
  train, valid = model_selection.train_test_split(files, test_size=0.15, random_state=13, shuffle=True)

  for file in train:
    shutil.copyfile(
        os.path.join(*[
                        path_to_source, 
                        'images', 
                        file
                      ]), 
        os.path.join(*[
                        path_to_destination, 
                        'images', 
                        'train', 
                        file
                      ]))
    shutil.copyfile(
        os.path.join(*[
                        path_to_source, 
                        'labels', 
                        file.replace('.jpg', '.txt')
                      ]), 
        os.path.join(*[
                        path_to_destination, 
                        'labels',
                        'train',
                        file.replace('.jpg', '.txt')
                      ]))
    
  for file in valid:
    shutil.copyfile(
        os.path.join(*[
                        path_to_source, 
                        'images', 
                        file
                      ]), 
        os.path.join(*[
                        path_to_destination, 
                        'images', 
                        'valid', 
                        file
                      ]))
    shutil.copyfile(
        os.path.join(*[
                        path_to_source, 
                        'labels', 
                        file.replace('.jpg', '.txt')
                      ]), 
        os.path.join(*[
                        path_to_destination, 
                        'labels',
                        'valid',
                        file.replace('.jpg', '.txt')
                      ]))


path_to_source = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\NIITO_Vertebrae_Dataset\\NIITO_Vertebrae_Dataset_Final_resized\\data_for_YOLO"
path_to_destination = "C:\\Users\\gieko\\Dropbox\\NIITO_Vertebrae\\yolov5\\data"
files = os.listdir(os.path.join(path_to_source, "images"))


data_split(files, path_to_source, path_to_destination)


In [ ]:
os.listdir(os.path.join(*[path_to_destination, "images", "valid"]))

NameError: name 'path_to_destination' is not defined

**Setting some augmentations**

In [ ]:
# !sed -i 's/perspective: 0.0/perspective: 0.1/g' ./yolov5/data/hyp.finetune.yaml
# !sed -i 's/shear: 0.0/shear: 0.1/g' ./yolov5/data/hyp.finetune.yaml
# !sed -i 's/flipud: 0.0/flipud: 0.5/g' ./yolov5/data/hyp.finetune.yaml
# !sed -i 's/degrees: 0.0/degrees: 0.2/g' ./yolov5/data/hyp.finetune.yaml

**Training**

--img 640 is the width of the images.  
`Dataset.yaml` file should be present in the directory pointed by --data.  
--cfg models/model.yaml is used to set the model we want to train on. I have used yolov5x.yaml, more information could be found [here.](https://github.com/ultralytics/yolov5#pretrained-checkpoints)  
  

Many useful tips and information regarding training and testing could be foung [here](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results). **This will helps to clear a good amount of doubts and errors. Also I recommend you to go through the issues section of the repo if you faces any errors or doubts. Legand has it that you will find the solution for your miseries there.**

In [ ]:
!python C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/train.py --img 640 --batch 4 --epochs 15 --device cpu --project C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/runs/NIITO_Vertebrae_YOLO --data C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/data/vertebrae.yaml --cfg C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/models/yolov5x.yaml --name test_run_Jan28_v5x
 
!python C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/train.py --img 640 --batch 4 --epochs 15 --device cpu --project C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/runs/NIITO_Vertebrae_YOLO --data C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/data/vertebrae.yaml --weights C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/yolov5s.pt --name test_run_Jan30_v5x_on_prerained_weights


!python C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/detect.py --source C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/data/images/test --weights C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/runs/NIITO_Vertebrae_YOLO/test_run_Jan28_v5x/weights/best.pt --data C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/data/vertebrae.yaml --max-det 26 --device cpu --hide-labels --hide-conf --classes 0 1 2 --line-thickness 2 --project C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/inference/NIITO_Vertebrae_YOLO/test_run_Feb1_v5s_on_prerained_weights

In [ ]:
!python C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/detect.py --source C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/data/images/test --weights C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/runs/NIITO_Vertebrae_YOLO/test_run_Jan30_v5x/weights/best.pt --data C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/data/vertebrae.yaml --max-det 26 --device cpu --hide-labels --hide-conf --classes 3 --line-thickness 2 --project C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/inference/NIITO_Vertebrae_YOLO/test_run_Jan30_v5x

detect: weights=['C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/runs/NIITO_Vertebrae_YOLO/test_run_Jan30_v5x/weights/best.pt'], source=C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/data/images/test, data=C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/data/vertebrae.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=26, device=cpu, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=[3], agnostic_nms=False, augment=False, visualize=False, update=False, project=C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/inference/NIITO_Vertebrae_YOLO/test_run_Jan30_v5x, name=exp, exist_ok=False, line_thickness=2, hide_labels=True, hide_conf=True, half=False, dnn=False, vid_stride=1
YOLOv5  2023-1-21 Python-3.9.0 torch-1.13.1+cpu CPU

Fusing layers... 
YOLOv5x summary: 322 layers, 86186872 parameters, 0 gradients, 203.8 GFLOPs
image 1/14 C:\Users\gieko\Dropbox\NIITO_Vertebrae\yolov5\data\images\test\001_SD.jpg: 640x288 (no detections), 665.8ms
image 2/14

In [ ]:
test_run_Feb22_v5x_on_prerained_weights

In [ ]:
!python C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/train.py --img 640 --batch 4 --epochs 5 --device cpu --project C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/runs/NIITO_Vertebrae_YOLO \
    --data C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/data/vertebrae.yaml --cfg C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/models/yolov5m.yaml --name test_run_Jan28_v5s

github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5


wandb: Currently logged in as: giekoolis. Use `wandb login --relogin` to force relogin
train: weights=..\yolov5\yolov5s.pt, cfg=C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/models/yolov5s.yaml, data=C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/data/vertebrae.yaml, hyp=..\yolov5\data\hyps\hyp.scratch-low.yaml, epochs=15, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=cpu, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=C:/Users/gieko/Dropbox/NIITO_Vertebrae/yolov5/runs/NIITO_Vertebrae_YOLO, name=test_run_Jan28_v5s, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5  2023-1-21 Python-3.9.0 torch-1.13.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, wei

In [ ]:
!python yolov5/train.py --img 640 --batch 16 --epochs 100 \
    --data /content/yolov5/tobacco_data.yaml --cfg /content/yolov5/models/yolov5n.yaml --name all-data-10thMay-cont --weights '/content/drive/MyDrive/Colab Notebooks/YOLO/weights/best.pt'

wandb: Currently logged in as: giekoolis. Use `wandb login --relogin` to force relogin
train: weights=/content/drive/MyDrive/Colab Notebooks/YOLO/weights/best.pt, cfg=/content/yolov5/models/yolov5n.yaml, data=/content/yolov5/tobacco_data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=all-data-10thMay-cont, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-177-gd059d1d torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, wei

**Testing**

**To predict images in a folder**  
--hide-labels is used to hide the labels in the detected images.  
--hide-conf is used to hide the confidence scores in the detected images.  --classes [0, 1, etc] used to detect only the classes mentioned here. For our use case we need only signature class, so use --classes 1.
--line-thickness integer used to set the thickness of bounging box.  
--save-crop and --save-txt used to save the crops and labels.  
--project could be used to specify the results path  

In [ ]:
!git clone -b test https://github.com/GiekoOlis/sgntrs /content/test/

Cloning into '/content/test'...
remote: Enumerating objects: 6196, done.
remote: Counting objects: 100% (1579/1579), done.
remote: Compressing objects: 100% (809/809), done.
remote: Total 6196 (delta 781), reused 1561 (delta 769), pack-reused 4617
Receiving objects: 100% (6196/6196), 726.92 MiB | 17.13 MiB/s, done.
Resolving deltas: 100% (1494/1494), done.


In [ ]:
!python yolov5/detect.py --source /content/test/yolo/images --weights '/content/drive/MyDrive/Colab Notebooks/YOLO/weights/best.pt' \
 --hide-labels --hide-conf --classes 1 --line-thickness 2 --project '/content/drive/MyDrive/Colab Notebooks/YOLO_after50more'

detect: weights=['/content/drive/MyDrive/Colab Notebooks/YOLO/weights/best.pt'], source=/content/test/yolo/images, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=[1], agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/Colab Notebooks/YOLO_after50more, name=exp, exist_ok=False, line_thickness=2, hide_labels=True, hide_conf=True, half=False, dnn=False
YOLOv5 🚀 v6.1-177-gd059d1d torch 1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1761871 parameters, 0 gradients, 4.2 GFLOPs
image 1/142 /content/test/yolo/images/000001.jpg: 640x384 1 DLSignature, Done. (0.013s)
image 2/142 /content/test/yolo/images/000028.jpg: 640x384 1 DLSignature, Done. (0.009s)
image 3/142 /content/test/yolo/images/000049 (2).jpg: 640x384 1 DLSignature, Done. (0.008s)
image 4/142 /content/t

**To pedict a single image**

In [ ]:
!python yolov5/detect.py --source tobacco_yolo_format/images/valid/imagename --weights 'runs/train/Tobacco-run/weights/best.pt' \
 --hide-labels --hide-conf --classes 1 --line-thickness 2 